In [ ]:
# Libraries for visualization
import pygame 
import math 

# Libraries for graphs
import sys
import numpy as np
import numpy.linalg as mat
import scipy as sp
import scipy.linalg as smat
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrow
from itertools import product 
from random import sample
import time
import warnings
warnings.filterwarnings('ignore')


"""""
Amazon floor visualization for single robot path planning

"""""

# Width (and height) of pop-up window for visualization 
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("Amazon Single Robot Path Planning Visualization: Shortest Path Alogrithm ")

# Colors for source, target, destination, and path
RED = (255, 0, 0)
GREEN = (0, 255, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (153,50,204)
GREY = (128, 128, 128)
DARK_GREEN = (0, 100, 0)

class Node:
    def __init__(self, row, col, width, total_rows): 
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width 
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows
        
    def get_pos(self): 
        return self.row, self.col
    
    def is_obstacle(self): 
        return self.color == BLACK 
    
    def is_source(self): 
        return self.color == DARK_GREEN
    
    def is_target(self): 
        return self.color == RED
    
    def reset(self): 
        self.color = WHITE
        
    def make_obstacle(self): 
        self.color = BLACK
    
    def make_source(self): 
        self.color = DARK_GREEN
    
    def make_target(self): 
        self.color = RED
        
    def make_reset(self):
        self.color = WHITE
        
    def make_path(self): 
        self.color = PURPLE
        
    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))
        
    # Draw arrow to augment a turn penatly
    # and provide a notion of orientation
    def draw_arrow(
        self,
        surface: pygame.Surface,
        start: pygame.Vector2,
        end: pygame.Vector2,
        body_width: int = 2,
        head_width: int = 4,
        head_height: int = 2,
    ):
        """Draw an arrow between start and end with the arrow head at the end.

        Args:
            surface (pygame.Surface): The surface to draw on
            start (pygame.Vector2): Start position
            end (pygame.Vector2): End position
            color (pygame.Color): Color of the arrow
            body_width (int, optional): Defaults to 2.
            head_width (int, optional): Defaults to 4.
            head_height (float, optional): Defaults to 2.
        """
        color = self.color
        self.color = GREY 

        arrow = start - end
        angle = arrow.angle_to(pygame.Vector2(0, -1))
        body_length = arrow.length() - head_height

        # Create the triangle head around the origin
        head_verts = [
            pygame.Vector2(0, head_height / 2),  # Center
            pygame.Vector2(head_width / 2, -head_height / 2),  # Bottomright
            pygame.Vector2(-head_width / 2, -head_height / 2),  # Bottomleft
        ]
        # Rotate and translate the head into place
        translation = pygame.Vector2(0, arrow.length() - (head_height / 2)).rotate(-angle)
        for i in range(len(head_verts)):
            head_verts[i].rotate_ip(-angle)
            head_verts[i] += translation
            head_verts[i] += start

        pygame.draw.polygon(surface, color, head_verts)

        # Stop weird shapes when the arrow is shorter than arrow head
        if arrow.length() >= head_height:
            # Calculate the body rect, rotate and translate into place
            body_verts = [
                pygame.Vector2(-body_width / 2, body_length / 2),  # Topleft
                pygame.Vector2(body_width / 2, body_length / 2),  # Topright
                pygame.Vector2(body_width / 2, -body_length / 2),  # Bottomright
                pygame.Vector2(-body_width / 2, -body_length / 2),  # Bottomleft
            ]
            translation = pygame.Vector2(0, body_length / 2).rotate(-angle)
            for i in range(len(body_verts)):
                body_verts[i].rotate_ip(-angle)
                body_verts[i] += translation
                body_verts[i] += start

            pygame.draw.polygon(surface, self.color, head_verts)

        
    def update_neighbors(self, grid): 
        # Check neighbors to ensure feasibility
        self.neighbors = []
        # Up
        if self.row > 0 and not grid[self.row - 1][self.col].is_obstacle():
            self.neighbors.append(grid[self.row - 1][self.col])
        # Down
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_obstacle(): 
            self.neighbors.append(grid[self.row + 1][self.col])
        # Left   
        if self.col > 0 and not grid[self.row][self.col - 1].is_obstacle(): 
            self.neighbors.append(grid[self.row][self.col - 1])
        # Right
        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_obstacle(): 
            self.neighbors.append(grid[self.row][self.col + 1])

        
          
    # Comparing nodes 
    def __lt__(self, other): 
        return False
    

# Create Grid 
def make_grid(rows, width): 
    grid = []
    # length of each node
    length = width // rows
    
    for i in range(rows):
        grid.append([])
        for j in range(rows): 
            node = Node(i, j, length, rows)
            grid[i].append(node) 
            
    return grid

# draw grid lines
def draw_grid(win, rows, width): 
    # length of each node in the grid
    length = width // rows
    
    for i in range(rows): 
        pygame.draw.line(win, GREY, (0, i * length), (width, i * length)) 
        
        for j in range(rows): 
            pygame.draw.line(win, GREY, (j * length, 0), (j * length, width))

# draw the grid using pygame
def draw(win, grid, rows, width): 
    # fill window screen with white 
    win.fill(WHITE)
    
    for row in grid: 
        for node in row: 
            node.draw(win)
            
    draw_grid(win, rows, width)
    pygame.display.update() 
    
    
def get_clicked_pos(pos, rows, width): 
    length = width // rows
    i, j = pos 
    # get which node mouse is on
    row = i // length
    col = j // length
    
    return row, col



"""""
Map visualization into Graph using NetworkX 

Applying Dijkstra's Algorithm 

"""""

# generate 2D graph 
def createGrid(n):
    
    G = nx.grid_2d_graph(n,n)
    pos = {(x,y):(y,-x) for x,y in G.nodes()}
      
    return G, pos

def plotGraph(G, pos, ROWS):
    
    node_size = 0
    if ROWS < 9: 
        node_size = 300
    elif 9 < ROWS < 15: 
        node_size = 200
    elif 15 < ROWS < 25: 
        node_size = 50
    elif ROWS > 25: 
        print("\n \n Graph too big to plot!!")
   
    print("\n NetworkX Graph: ")
    nx.draw_networkx(G, pos=pos, 
                node_color='lightgreen', 
                with_labels=False,
                node_size=node_size)

    plt.axis('off')
    plt.show()
    
    

# Assign edge costs, random if no dict given
def edgeCosts(G, params = {}):
    
    # if edge costs given in params dict. then assign them
    # else assign random weights
    if 'edge_costs' in params:
        edge_costs_list = params['edge_costs'];
    else:
        edge_costs_list = 4*np.random.rand(len(G.edges)) 
        
    edge_costs = {k:v for k,v in zip(G.edges, edge_costs_list)}
    nx.set_edge_attributes(G, edge_costs, 'c')
    
    return edge_costs



def dijkstra_algorithm(draw, G, grid, source, target, obstacles): 
    
    """""
    draw - draw function for visualization 
    G - NetworkX graph 
    grid - visualization grid 
    source - starting node
    target - ending node
    obstacles - obstacle nodes
    
    """""
    
    
    obstacle_dijkstra = obstacles
    
    # Find all edges connected to obstacles       
    edges_obstacles = []
    for i in obstacle_dijkstra: 
        edges_obstacles.append(G.edges(i))
        # find all pairs for each node                
        for j in edges_obstacles:  
            edge_nodes = j
            # find each edge pair            
            for k in edge_nodes: 
                u, v = k
                # Hide edges connected to obstacle nodes with 'None'
                nx.set_edge_attributes(G, {(u, v): {"weight": None}})
                
                
    # Print all edges in graphs with their weights
    #for u, v, d in G.edges(data=True):
        #print(f"({u}, {v}), {d=}")
                                                      
    # Find which nodes have obstacles
    nodes_list = list(G.nodes())
    st = set(obstacle_dijkstra)
    nodes_index = [i for i, e in enumerate(nodes_list) if e in st]
     
    # Random edge costs
    random_edgeCosts = edgeCosts(G, params = {})
    
    # Dijkstra from NetworkX
    start = time.time()
    path = nx.dijkstra_path(G, source, target)  
    end = time.time()
    time_complexity = round(end - start, 4)
    time_complexity *= 1000 # milliseconds
      
    # Draw path on grid
    for index in path[1:-1]:
        col, row = index
        node = grid[row][col] 
        node.make_path()
        draw() 
        
    return path, time_complexity
    
    
def main(win, width): 
    # nxn grid of the floor (grid) 
    ROWS = 18
    grid = make_grid(ROWS, width)
    # length of each node
    length = width // ROWS
    
    obstacle_list = []
    
    source = None
    target = None 
    
    run = True 
    
    while run: 
        draw(win, grid, ROWS, width)
        
        
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT: 
                run = False
                
            # get left click pos
            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width) 
                node = grid[row][col] 
                print(col, row)
                print(pos)
                
                # set source, target, and obstacles with left click
                if not source and node != target:
                    source_dijkstra = (col, row)
                    #print("Source: ", source_dijkstra)
                    source = node
                    source.make_source()
                    screen_row = col * length
                    screen_col = row * length
                    start = pygame.Vector2(screen_row, screen_row + (length / 2))
                    end = pygame.Vector2(screen_col + length, screen_row + (length / 2))
                    
                    #node.draw_arrow(win, start, end, 1e-20, 1e-20, 1e-20)
                    
                elif not target and node != source:
                    target_dijkstra = (col, row)
                    #print("Target: ", target_dijkstra) 
                    target = node
                    target.make_target()
                    
                elif node != target and node != source:
                    obs = (col, row) 
                    obstacle_list.append(obs)
                    node.make_obstacle() 
                    
            
            # get right click pos
            elif pygame.mouse.get_pressed()[2]: 
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width) 
                node = grid[row][col] 
                node.reset()
                # if right click then reset node
                if node == source: 
                    source = None
                
                elif node == target: 
                    target = None
                    
            # Pressing space will start the algorithm       
            if event.type == pygame.KEYDOWN: 
                if event.key == pygame.K_SPACE and source and target: 
                    for row in grid:
                        for node in row: 
                            node.update_neighbors(grid)
                            
                    G, pos = createGrid(ROWS)  
                            
                    # Get obstacle positions with no duplicates
                    obstacle_dijkstra = []
                    for i in obstacle_list: 
                        if i not in obstacle_dijkstra:
                            obstacle_dijkstra.append(i)
                    
                    path, time_complexity = dijkstra_algorithm(lambda: draw(win, grid, ROWS, width),
                                                G, grid,source_dijkstra, target_dijkstra, obstacle_dijkstra)
                    
                    print("\n Algorithm Run Time: ", time_complexity, "ms")
                    
                    #print("\n Total Cost: \n ", cost)
                       
                    print("\n Dijkstra's Shortest Path: \n \n ", path) 
                    
                    
        
                    """""
                    Plot NetworkX Graph
                    
                    """""
                    #plotGraph(G, pos, ROWS)
                
                    
                # Press 'c' key to reset the program    
                if event.key == pygame.K_c: 
                    source = None
                    target = None
                    grid = make_grid(ROWS, width)
                
                
                
    pygame.quit()
    
    
# Run program
main(WIN, WIDTH)          
        
    
    
    


1 1
(66, 69)
16 16
(730, 740)
16 16
(730, 740)
3 2
(103, 162)
3 2
(102, 162)
3 2
(101, 161)
3 2
(100, 161)
3 2
(99, 160)
3 2
(97, 159)
3 2
(96, 159)
3 2
(95, 158)
3 2
(93, 158)
3 2
(91, 158)
3 2
(90, 158)
3 2
(88, 158)
3 1
(87, 158)
3 1
(86, 158)
3 1
(85, 158)
3 1
(84, 158)
3 1
(83, 158)
3 1
(82, 159)
3 1
(81, 161)
3 1
(81, 162)
3 1
(80, 165)
3 1
(80, 166)
3 1
(79, 169)
3 1
(79, 172)
3 1
(79, 174)
4 1
(79, 177)
4 1
(79, 181)
4 1
(78, 184)
4 1
(78, 187)
4 1
(78, 189)
4 1
(78, 192)
4 1
(78, 194)
4 1
(78, 195)
4 1
(78, 197)
4 1
(78, 198)
4 1
(78, 199)
4 1
(78, 200)
4 1
(78, 201)
4 1
(79, 201)
4 1
(81, 202)
4 1
(82, 203)
4 1
(84, 203)
4 1
(86, 203)
4 2
(88, 204)
4 2
(90, 204)
4 2
(92, 205)
4 2
(94, 205)
4 2
(96, 205)
4 2
(97, 206)
4 2
(99, 206)
4 2
(100, 206)
4 2
(101, 207)
4 2
(102, 207)
4 2
(103, 207)
4 2
(103, 206)
4 2
(104, 205)
4 2
(104, 204)
4 2
(104, 202)
4 2
(105, 201)
4 2
(105, 200)
4 2
(106, 199)
4 2
(106, 198)
4 2
(106, 197)
4 2
(107, 197)
4 2
(107, 196)
4 2
(108, 196)
1 7
(330,

15 6
(305, 684)
15 6
(304, 687)
15 6
(304, 691)
15 6
(303, 695)
15 6
(303, 701)
16 6
(303, 707)
16 6
(303, 712)
16 6
(303, 718)
16 6
(303, 721)
16 6
(303, 725)
16 6
(303, 730)
16 6
(303, 735)
16 6
(303, 738)
16 6
(303, 740)
16 6
(303, 742)
16 6
(304, 743)
16 6
(305, 745)
16 6
(306, 746)
16 7
(308, 747)
17 7
(311, 748)
17 7
(312, 748)
17 7
(314, 748)
17 7
(316, 748)
17 7
(318, 748)
17 7
(320, 748)
17 7
(322, 748)
17 7
(324, 748)
16 7
(326, 747)
16 7
(328, 745)
16 7
(329, 744)
16 7
(331, 742)
16 7
(332, 740)
16 7
(333, 739)
16 7
(334, 738)
16 7
(335, 737)
16 7
(335, 735)
16 7
(336, 734)
16 7
(337, 733)
16 7
(337, 732)
16 7
(337, 731)
16 7
(338, 730)
16 7
(338, 728)
16 7
(338, 727)
16 7
(339, 725)
16 7
(339, 724)
16 7
(339, 722)
16 7
(339, 721)
16 7
(339, 720)
16 7
(339, 719)
16 7
(339, 718)
16 7
(339, 717)
16 7
(338, 716)
16 7
(338, 715)
16 7
(338, 714)
16 7
(337, 714)
16 7
(337, 713)
16 7
(336, 713)
16 7
(336, 712)
16 7
(335, 712)
16 7
(334, 712)
16 7
(333, 712)
14 11
(507, 647)
14 11
(